### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    97935516ADA083B7A1F13BD9D89EBC11 0803050A0502040107030600030504...30506060B0A060205
```    
where:
```
    97935516ADA083B7A1F13BD9D89EBC11 is a ciphertext
    
    0803050A0502040107030600030504...30506060B0A060205 is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace consists of a random data plus at certain position a trace contains Hamming weights of bytes for a 10th round input state of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round. Those Hamming weights contain also some noise (they are not perfectly measured).

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [1]:
import numpy as np
import sca_training
import matplotlib.pyplot as plt

def hex_int(hex):
    return np.array(list(bytearray.fromhex(hex)))

ciphers = np.load('ciphers.npy')
hws = np.load('traces.npy')

print(ciphers.shape)
print(hws.shape)

(16, 200)
(100, 200)


In [9]:
k10 = [0]*16
threshold_pcc = 0.7

def matrix_corrcoef(matrix, vector):
    return np.array([abs(np.corrcoef(vector, row)[0][1]) for row in matrix])

for i in range(0,16):
    print(i)
    pcc = 0
    k_max = 0
    for k in range(256):
        cand = sca_training.HW_uint8[sca_training.invSbox[np.bitwise_xor(ciphers[i], k)]]
        corr = matrix_corrcoef(hws, cand)
        # plt.plot(corr)
        # break
        corr = max(corr)
        if corr > pcc:
            pcc = corr
            k_max = k
            print(pcc)
            if pcc > threshold_pcc:
                break
    # break
    k10[i] = k_max

0
0.20592377876297985
0.2129039738983797
0.21655436747472134
0.23205889122752296
0.27000785260555177
0.2790407026035343
0.7476736796407021
1
0.17169765343204857
0.17601134090801374
0.21308835173030782
0.22408187356854603
0.22494165012759665
0.23180572852540665
0.2581710432764512
0.29090724673480983
0.8090579879649106
2
0.15264105129430416
0.16318827718844248
0.16474280430393481
0.17034731154964045
0.20411407090685138
0.22437704531506425
0.23459404571321296
0.24365107749206633
0.2589292307862379
0.7686944654654901
3
0.18154354606854137
0.20604398616780412
0.24807502229329043
0.2697571547263594
0.27933685533513947
0.7958678793348697
4
0.2513607014482879
0.2748270572684384
0.3061161753290586
0.718215455013535
5
0.1963043069389505
0.20116758531581044
0.2168741142625253
0.23518294238486925
0.25252825234729526
0.2853698176086739
0.2884144755414833
0.7436394197961764
6
0.22589928665547396
0.23240872378901098
0.2542018604689676
0.2997290500255797
0.7680646507352259
7
0.20334631199868833
0.2205

In [8]:
print(k10)
mk = sca_training.inverse_key_expansion(np.array(k10))[0][0]
flag = ''.join(chr(c) for c in mk)
flag

[34, 67, 111, 160, 41, 208, 225, 54, 236, 173, 94, 30, 128, 139, 203, 151]


'HEIG{Iamhere-54}'

### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)

In [ ]:
import numpy as np
import sca_training

def hex_int(hex):
    return np.array(list(bytearray.fromhex(hex)))

In [13]:
test = [[] for _ in range(16)]
test[0].append(1)
print(test)

[[1], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
